In [2]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import skeletonDefMixamo as skd
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm

from PhaseFunctionedNetwork import PhaseFunctionedNetwork

# set seeds for reproduceability
torch.manual_seed(42)
np.random.seed(42)
rng = np.random.RandomState(42)

In [2]:
# load data
# X = np.float32(np.loadtxt('./data/Input.txt'))
# Y = np.float32(np.loadtxt('./data/Output.txt'))
# P = np.float32(np.loadtxt('./data/Phases.txt'))
# print(X.shape, Y.shape, P.shape)

In [3]:
def get_phases(X_input_arr):
    phase = np.linspace(0, 2*np.pi, num=X_input_arr.shape[0])
    return phase

In [7]:
# Process Data (Note currently this data is NOT MIRRORD, CAN DOUBLE WITH MIRRORING)

# Idle Data
X_idle = np.float32(np.loadtxt('C:/Users/Ana/Desktop/dev/pfnn-dev/Export/Input_Idle.txt'))
Y_idle = np.float32(np.loadtxt('C:/Users/Ana/Desktop/dev/pfnn-dev/Export/Output_Idle.txt'))
P_idle = get_phases(X_idle)
print(f"Idle data shape: {X_idle.shape}, {Y_idle.shape}, {P_idle.shape}")

# Big Forward Jump
X_jump_for_BIG = np.float32(np.loadtxt('C:/Users/Ana/Desktop/dev/pfnn-dev/Export/Input_jump_for_BIG.txt'))
Y_jump_for_BIG = np.float32(np.loadtxt('C:/Users/Ana/Desktop/dev/pfnn-dev/Export/Output_jump_for_BIG.txt'))
P_jump_for_BIG = get_phases(X_jump_for_BIG)
print(f"Jump Forward BIG data shape: {X_jump_for_BIG.shape}, {Y_jump_for_BIG.shape}, {P_jump_for_BIG.shape}")

# Small Forward Jump
X_jump_for_SMALL = np.float32(np.loadtxt('C:/Users/Ana/Desktop/dev/pfnn-dev/Export/Input_jump_for_SMALL.txt'))
Y_jump_for_SMALL = np.float32(np.loadtxt('C:/Users/Ana/Desktop/dev/pfnn-dev/Export/Output_jump_for_SMALL.txt'))
P_jump_for_SMALL = get_phases(X_jump_for_SMALL)
print(f"Jump Forward SMALL data shape: {X_jump_for_SMALL.shape}, {Y_jump_for_SMALL.shape}, {P_jump_for_SMALL.shape}")

Idle data shape: (995, 888), (995, 819), (995,)
Jump Forward BIG data shape: (169, 888), (169, 819), (169,)
Jump Forward SMALL data shape: (112, 888), (112, 819), (112,)


In [8]:
# Concatenate Data into indivudal X, Y, P Arrays
X = np.concatenate((X_idle, X_jump_for_BIG, X_jump_for_SMALL))
print(f"X: {X.shape}")

Y = np.concatenate((Y_idle, Y_jump_for_BIG, Y_jump_for_SMALL))
print(f"Y: {Y.shape}")

P = np.concatenate((P_idle, P_jump_for_BIG, P_jump_for_SMALL))
print(f"P: {P.shape}")

# Debugging only use idle
# X = X_idle
# Y = Y_idle
# P = P_idle

X: (1276, 888)
Y: (1276, 819)
P: (1276,)


In [9]:
# Phase logic
delta_phase = P[1:] - P[:-1]
delta_phase[delta_phase < 0] = (1.0 - P[:-1] + P[1:])[delta_phase < 0]
delta_phase = np.append(delta_phase, 0)

Y = np.concatenate([Y, delta_phase [..., np.newaxis]], axis=-1)

# calculate mean and std
Xmean, Xstd = X.mean(axis=0), X.std(axis=0)
Ymean, Ystd = Y.mean(axis=0), Y.std(axis=0)

for i in range(Xstd.size):
    if (Xstd[i]==0):
        Xstd[i]=1
for i in range(Ystd.size):
    if (Ystd[i]==0):
        Ystd[i]=1

# normalize data
X = (X - Xmean) / Xstd
Y = (Y - Ymean) / Ystd

# save mean / std / min / max

Xmean.astype(np.float32).tofile('./weights/Xmean.bin')
Ymean.astype(np.float32).tofile('./weights/Ymean.bin')
Xstd.astype(np.float32).tofile('./weights/Xstd.bin')
Ystd.astype(np.float32).tofile('./weights/Ystd.bin')

In [10]:
# append phase as additional feature
input = torch.tensor(np.concatenate([X, P [..., np.newaxis]], axis=-1))
target = torch.tensor(Y)

print(f"Input shape {input.shape}")
print(f"Target shape {target.shape}")

dataset = TensorDataset(input, target)

BATCH_SIZE = 32
train_dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

Input shape torch.Size([1276, 889])
Target shape torch.Size([1276, 820])


In [11]:
# ensure GPU is available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f"Using device: {device}")
device = "cuda"

# custom loss function
def loss_func(output, target, model):
    loss = torch.mean((output - target)**2) + model.cost()
    return loss

In [12]:
model = PhaseFunctionedNetwork(input_shape=input.shape[1], output_shape=target.shape[1])
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001)

epochs=5

for epoch in range(epochs):
    model.train()
    loss_list = []
    for i, batch in enumerate(tqdm(train_dataloader)):
        input, target = batch
        input, target = input.to(device), target.to(device)

        # forward pass
        output = model(input)
        loss = loss_func(output, target, model)
        loss_list.append(loss.item())

        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {np.average(loss_list)}')


100%|██████████| 40/40 [00:03<00:00, 10.85it/s]


Epoch [1/5], Loss: 0.9844809290604974


100%|██████████| 40/40 [00:03<00:00, 11.45it/s]


Epoch [2/5], Loss: 0.8015381094889669


100%|██████████| 40/40 [00:03<00:00, 11.49it/s]


Epoch [3/5], Loss: 0.6970964094246226


100%|██████████| 40/40 [00:03<00:00, 11.53it/s]


Epoch [4/5], Loss: 0.6673806187058976


100%|██████████| 40/40 [00:03<00:00, 11.53it/s]

Epoch [5/5], Loss: 0.6388783828781259


In [13]:
# save weights
model.precompute_and_save_weights()